In [1]:
import numpy as np
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from tqdm import tqdm

from reconstruct import get_SIFT_keypoints, get_X
from utils import read, compute_F
from stereo import Stereo

In [2]:
def select_X(candidates):
    pd = pairwise_distances(candidates)
    
    ratings = np.sum(1/(pd.mean()/2+pd), axis=1)
    return candidates[np.argmax(ratings)]

In [3]:
images_number = [i for i in range(27)]
n_images = len(images_number)

In [ ]:
images, P, silhouettes = zip(*[read(number) for number in images_number])

In [ ]:
P = np.array(P)
silhouettes = np.array(silhouettes)

In [ ]:
F = np.zeros((n_images, n_images, 3, 3))

In [7]:
for i in range(n_images):
    for j in range(i+1, n_images):
        f = compute_F(P[i], P[j])
        F[i,j] = f
        F[j,i] = f

In [8]:
n_keypoints = 40

In [9]:
kps = get_SIFT_keypoints(images[0], silhouettes[0], n_keypoints)

In [10]:
KP = np.zeros((n_images, n_keypoints, 3))

In [11]:
for i, (image, silhouette) in enumerate(zip(images, silhouettes)):
    KP[i] = get_SIFT_keypoints(image, silhouette, n_keypoints_min=n_keypoints)[:n_keypoints]

In [13]:
X_candidates = np.zeros((n_images, n_keypoints, n_images, 3))

In [14]:
stereo = Stereo(silhouettes.shape[2], silhouettes.shape[1])

In [15]:
for i in tqdm(range(n_images)):
    for ik, x1 in enumerate(KP[i]):
        for j in range(n_images):
            if j != i:
                x2 = stereo.find_x2(x1, F[i,j], images[i], images[j], kernel_size=[20,20])
                if x2 is not None:
                    X = get_X(P[i], P[j], x1, x2)
                    X_candidates[i][ik][j] = X[:-1]

  4%|▎         | 1/27 [00:07<03:23,  7.84s/it]/home/rockl33/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
 15%|█▍        | 4/27 [01:12<06:51, 17.88s/it]/home/rockl33/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
 19%|█▊        | 5/27 [01:40<07:37, 20.81s/it]/home/rockl33/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
/home/rockl33/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))
 22%|██▏       | 6/27 [02:10<08:16, 23.63s/it]/home/rockl33/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:1166: RuntimeWarning: div

In [16]:
X_candidates.resize((n_images*n_keypoints, n_images, 3))

In [17]:
X_list = []

for candidates in X_candidates:
    candidates = np.array([c for c in candidates if np.sum(c) != 0])
    if len(candidates) > 0:
        X_list.append(select_X(candidates))
    
X_list = np.array(X_list)

In [20]:
#%matplotlib inline 
%matplotlib widget
fig = plt.figure()
ax = Axes3D(fig)

X_list = np.array(X_list)
ax.scatter(X_list[:,0], X_list[:,1], X_list[:,2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
ax.plot_trisurf(X_list[:,0], X_list[:,1], X_list[:,2])